# Recipe Rating Machine Learning

In [ ]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sqlite3
from sqlite3 import Error

# Dataset:  epicurious_ratings.csv

Source: Kaggle (https://www.kaggle.com/hugodarwood/epirecipes#full_format_recipes.json)

Recipe information lifted from: http://www.epicurious.com/recipes-menus

Description: Recipe data information, containing categorical and nutritional information

Variables/Columns  
TITLE  
RATING: 0.000 to 5.000  
  (0.0 - 0.999 = Eww! / 1.0 - 1.999 = Better than not eating / 2.0 - 2.999 = Edible / I guess, 3.000 - 3.999 = Pretty darn good / 4.000 - 4.999 = Git in ma belleh)
CALORIES   
FAT  
SODIUM


## Potential future, interesting work to add in...  
=> Distribution of ratings..  
  => How many ingredients per recipe and does it correlate to an increase in star ratings  
=> Locate columns with counts of "1"s greater than 1000 (to identify specialty diets and ingrediants) ? Do a for loop and summation  


## Questions...


optiomal 
Normal linear regression
Use each variable in a neural network... 
Keras 

In [ ]:
# Read the csv file into a pandas DataFrame
ratings = pd.read_csv('Resources/epicurious_ratings.csv')
ratings

In [ ]:
ratings.shape

In [ ]:
# Create new dataframe with NaN values removed
ratings = ratings.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
ratings

In [ ]:
# Reset the index on the new dataframe
ratings.reset_index(drop=True)

In [ ]:
# write this back to csv and for upload to gitHub
ratings.to_csv(r"C:\Users\Deanna\Desktop\Project3\Recipe_Rating\Resources\epi_ratings_clean.csv")  

In [ ]:
keep_cols = ["title", "rating", "calories", "protein", "fat", "sodium"]
# Potential side chart / analysis - speciality diet ratings / specific ingrediants include: "vegan", "vegetarian", "dairy free", "wheat/gluten-free", 
# "peanut free", "meat", "alcoholic", "chocolate", "cheese"

In [ ]:
ratings = ratings[keep_cols]
ratings

In [ ]:
ratings.shape